In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from scipy.stats import pearsonr
from scipy.stats import kendalltau

In [2]:
All_data_names = ['Reaction_CO_1.5h', 'Reaction_CO_biphenyl', 'Reaction_CO_ortho', 'Reaction_CO_Cl',
                  'Reaction_CS', 'Reaction_CN', 'Reaction_2+2',
                  'Reaction_CF3', 'Reaction_CH2CF3', 'Reaction_CH2F', 'Reaction_Cy', 'Reaction_SCF3',
                  'Reaction_OCH2F', 'Reaction_P', 'Reaction_Si']

train_size_list = [15, 20, 30]

In [3]:
for target in All_data_names:
    for train_size in train_size_list:
        data = pd.read_csv(f'../../data/data_woTL/data_60_{target}.csv')
        y = pd.DataFrame(data['Yield'],columns=['Yield'])
        X = data.drop(columns=['Name', 'ID', 'Yield'])
    
        r2_train = []
        r2_test = []
        pearson_train = []
        pearson_test = []
        kendalltau_train = []
        kendalltau_test = []
        for i in range(0,100):
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=i)
            reg = RandomForestRegressor(random_state=0)
            reg.fit(X_train,y_train['Yield'])
            y_pred1 = reg.predict(X_train)
            y_pred2 = reg.predict(X_test)
            r2_train.append(metrics.r2_score(y_train, y_pred1))
            r2_test.append(metrics.r2_score(y_test, y_pred2))
            pearson_train.append(pearsonr(y_train.values.ravel(), y_pred1.ravel())[0])
            pearson_test.append(pearsonr(y_test.values.ravel(), y_pred2.ravel())[0])
            kendalltau_train.append(kendalltau(y_train.values.ravel(), y_pred1.ravel())[0])
            kendalltau_test.append(kendalltau(y_test.values.ravel(), y_pred2.ravel())[0])
    
        r2_train = pd.DataFrame(data=r2_train, columns=['r2_train'])
        r2_test = pd.DataFrame(data=r2_test, columns=['r2_test'])
        pr_train = pd.DataFrame(data=pearson_train, columns=['pr_train'])
        pr_test = pd.DataFrame(data=pearson_test, columns=['pr_test'])
        kt_train = pd.DataFrame(kendalltau_train, columns=['kt_train'])
        kt_test = pd.DataFrame(kendalltau_test, columns=['kt_test'])
        result = pd.concat([r2_train, r2_test, pr_train, pr_test, kt_train, kt_test], axis=1, join='inner')
        result.to_csv(f'../results_reg/RF/train_size_{train_size}/result_{target}.csv')